In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)y
import random
import json
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/distance-opt-km-json/Distance_OPT_KM.json


In [2]:
!pip install torch transformers transformer_lens 
!pip install -i https://pypi.org/simple/ bitsandbytes

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.9/175.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.10.0 requires scipy<1.14,>=1.4.1, but you have scipy 1.14.1 which is incompatible.
ydata-profiling 4.10.0 requires typeguard<5,>=3, but you have typeguard 2.13.3 which is incompatible.
Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.4 MB/s eta 0:00:0000:0100:01


In [ ]:
HF_token ="<token>"

In [4]:
use_8bit = True

bnb_config = BitsAndBytesConfig(
    load_in_8bit=use_8bit,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)

# Load the Llama 3 8B model and tokenizer
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"  
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_token)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
llama_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                   quantization_config=bnb_config,
                                                   device_map="auto",
                                                   token=HF_token)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [6]:
system={"role": "system", 
    "content": 
"""You are an expert in telecommunication systems. Your task is to classify the user request to one of the five optimization classes.

Class names and Objectives are as follows:
1. KM (K-Means): Distance Reduction
2. WKHM (Weighted K-Harmonic Means): Balancing Load and Power Loss Optimization
3. CKM (Constrained K-Means): Load Balancing 
4. KC (K-Centers): Maximize Minimum RSRP[Received Signal Reference Power]
5. KHM (K-Harmonic Means): Power Loss Optimization

IMPORTANT RULES:
1. Provide reasoning for your choice of class.
2. For the final answer provide the class name within <> example <KM>, <WKHM>.
3. Start your Response with a "^" symbol.
"""
}


In [7]:
with open("/kaggle/input/distance-opt-km-json/Distance_OPT_KM.json") as f:
    Distance_OPT=json.load(f)

In [ ]:
for i in range(3):
    for j in range(40):
        request=Distance_OP T[str(i)][j]
        message=[system,
            {"role":"user","content":request}]
        input_ids = tokenizer.apply_chat_template(message, return_tensors="pt",padding=True, truncation=True,).to(llama_model.device)
    
        with torch.no_grad():
            output = llama_model.generate(input_ids, max_length=input_ids.shape[1]+500, num_return_sequences=1, temperature=1.25);
    
        modified_prompt = tokenizer.decode(output[0], skip_special_tokens=True)
        print(modified_prompt.split("^")[-1].strip())
        

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


To determine the optimization class for this problem, let's break it down into its essential components.

1. The goal here is to minimize the distance between users and routers (distance reduction), which aligns with the objective of Class 1: <KM>.
2. We also consider optimizing user distribution across the banquet hall following specific patterns. However, this seems more of an input factor rather than an objective in itself.
3. The path loss, distribution of the users and their respective locations suggest a direct influence from both the location of users (150) and the routers (8) and how they would have an effect on power coverage but the overall problem stated can be simplified as reducing the minimum distance.

Given the above considerations, I would classify this problem as belonging to Class 1: <KM>.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Given the requirements, let's break down the problem. The task is to position 4 Huawei NE40 routers to minimize the distance to 80 uniformly spread users along the X-axis and bi-exponentially spread along the Y-axis in a 40m x 15m office with a height of 3m.

To classify this request into one of the optimization classes, we must consider the objective of the task.

1. The task involves minimizing the distance to users, which aligns with the 'Distance Reduction' objective of the KM (K-Means) class.
2. The presence of bi-exponential distribution along the Y-axis and different user distributions might require a more complex model to capture these characteristics.

However, given that the primary objective of minimizing distance to users is in line with the KM class, it can be argued that this request is best categorized under the 'KM' class.

< KM >


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Based on the information provided, it seems like you're trying to optimize the placement of Huawei NE40 routers to minimize the distance between each user and the nearest router in the 55m x 25m office with a 5m height.

Considering the specifics, such as the path loss parameters and the bi-exponential user distribution along the X-axis and exponential along the Y-axis, it would be suitable to apply the K-Means algorithm to optimize the router placement. The objective here is to minimize the distance, which aligns perfectly with the "Distance Reduction" objective of the K-Means algorithm (Class KM).

<KM>


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


This problem seems to involve minimizing the distance between users and routers (also known as base stations), which is a classic problem in cellular network optimization.

Since we want to minimize the distance, this problem falls under the category of optimizing for Distance Reduction. 

Therefore, the most suitable class for this problem is <KM>.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The user request involves optimizing the distance between 50 users in a given room and considering path loss parameters. This suggests that minimizing the distance or power loss is a key consideration. 

The mention of Gaussian distribution along both axes for the users further implies a need to balance the distance and optimize the placement of routers and users to minimize interference and maximize coverage.

The presence of 3 Nokia 7750 routers with a specified transmit power of 22 dBm, which we can convert to Watts as approximately 199.526 milliwatts, further indicates that power loss and balancing load between the routers should be considered.

Considering these factors, the optimization goal seems to focus on load balancing and power loss optimization. 

Hence, the appropriate class is <WKHM>. The Weighted K-Harmonic Means algorithm takes into account load and power loss while considering an optimal placement strategy that maximizes efficiency, which seems to align with the scena